# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import itertools
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime as dt, date, timedelta

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"


## Generate Cities List

In [36]:
import numpy as np
#Define our lat/lon range and increments between the range
min_lat = 32
max_lat = 42
min_lon = -114
max_lon = -125
increment = .3

#Build our lat/lon list within range
lat_lons = []

lats = [round(num, 2) for num in np.arange(min_lat, max_lat, increment).tolist()]
lons = [round(num, 2) for num in np.arange(min_lon, max_lon, -(increment)).tolist()]
lat_lons = list(itertools.product(lats, lons))
len(lat_lons)


1258

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [37]:
import datetime
day_of_eq = "2020-08-08"
day_of_eq_dtformat = dt.strptime(day_of_eq,'%Y-%m-%d')
day_before_eq_dtformat = day_of_eq_dtformat - datetime.timedelta(days=1)
day_before_eq=dt.strftime(day_before_eq_dtformat,'%Y-%m-%d')
day_before_eq

'2020-08-07'

In [13]:
appid = weather_api_key

In [38]:
base_url = "http://api.weatherstack.com/historical?"

counter = 1
city_data = []

for lat_lon in lat_lons:
    params = {
      "query" : f'{lat_lon[0]},{lat_lon[1]}',
      "historical_date_start" : day_before_eq,
      "historical_date_end" : day_of_eq,  
      "hourly" : 0,
      "units" : "f",
      "access_key" : appid
       }

    weather = requests.get(base_url, params=params).json()
    
    lat = lat_lon[0]
    lon = lat_lon[1]
    temp1 = weather["historical"][day_of_eq]["avgtemp"]
    temp0 = weather["historical"][day_before_eq]["avgtemp"]
    date_of_eq = weather["historical"][day_of_eq]["date"]
    date_before_eq = weather["historical"][day_before_eq]["date"]
    city = weather["location"]["name"]
    country = weather["location"]["country"]

    city_data.append({"City" : city
                    ,"Country" : country
                    ,"Latitude" : lat
                    ,"Longitude" : lon
                    ,f'Avg Temperature {date_before_eq}': temp0
                    ,f'Avg Temperature {date_of_eq}': temp1
                    })

    print(f"Processing Record {counter} - {city}")
    counter = counter + 1


Processing Record 1 - El Pedregal
Processing Record 2 - La Joya
Processing Record 3 - La Capital Del Desierto
Processing Record 4 - El Doctor
Processing Record 5 - El Pokar
Processing Record 6 - El Carrizo
Processing Record 7 - El Escondido
Processing Record 8 - La Huertita
Processing Record 9 - El Campito
Processing Record 10 - El Capricho
Processing Record 11 - Sonorabampo
Processing Record 12 - Marisol
Processing Record 13 - Isla Coronado
Processing Record 14 - Isla Coronado
Processing Record 15 - Isla Coronado
Processing Record 16 - Isla Coronado
Processing Record 17 - Isla Coronado
Processing Record 18 - Avalon
Processing Record 19 - Avalon
Processing Record 20 - Avalon
Processing Record 21 - None
Processing Record 22 - None
Processing Record 23 - None
Processing Record 24 - None
Processing Record 25 - None
Processing Record 26 - None
Processing Record 27 - None
Processing Record 28 - None
Processing Record 29 - None
Processing Record 30 - None
Processing Record 31 - None
Processi

In [39]:
print(weather)

{'request': {'type': 'LatLon', 'query': 'Lat 41.90 and Lon -124.80', 'language': 'en', 'unit': 'f'}, 'location': {'name': 'Crescent City', 'country': 'United States of America', 'region': 'California', 'lat': '41.756', 'lon': '-124.201', 'timezone_id': 'America/Los_Angeles', 'localtime': '2020-08-15 02:12', 'localtime_epoch': 1597457520, 'utc_offset': '-7.0'}, 'current': {'observation_time': '09:12 AM', 'temperature': 63, 'weather_code': 143, 'weather_icons': ['https://assets.weatherstack.com/images/wsymbols01_png_64/wsymbol_0006_mist.png'], 'weather_descriptions': ['Haze'], 'wind_speed': 0, 'wind_degree': 0, 'wind_dir': 'N', 'pressure': 1012, 'precip': 0, 'humidity': 80, 'cloudcover': 0, 'feelslike': 63, 'uv_index': 1, 'visibility': 6, 'is_day': 'no'}, 'historical': {'2020-08-07': {'date': '2020-08-07', 'date_epoch': 1596758400, 'astro': {'sunrise': '06:19 AM', 'sunset': '08:29 PM', 'moonrise': '10:55 PM', 'moonset': '10:13 AM', 'moon_phase': 'Waning Gibbous', 'moon_illumination': 74}

In [40]:
df = pd.DataFrame(city_data)

In [41]:
df['Temp_Difference']=abs(df[f'Avg Temperature {date_of_eq}'] - df[f'Avg Temperature {date_before_eq}'])
df

,City,Country,Latitude,Longitude,Avg Temperature 2020-08-07,Avg Temperature 2020-08-08,Temp_Difference
0,El Pedregal,Mexico,32.0,-114.0,93,97,4
1,La Joya,Mexico,32.0,-114.3,91,93,2
2,La Capital Del Desierto,Mexico,32.0,-114.6,91,95,4
3,El Doctor,Mexico,32.0,-114.9,91,95,4
4,El Pokar,Mexico,32.0,-115.2,91,95,4
...,...,...,...,...,...,...,...
1253,Happy Camp,United States of America,41.9,-123.6,79,84,5
1254,Patrick Creek,United States of America,41.9,-123.9,68,72,4
1255,Douglas Park,United States of America,41.9,-124.2,64,64,0
1256,Crescent City,United States of America,41.9,-124.5,64,64,0


In [42]:
df.to_json(r'C:\Users\rona_\Desktop\Aug08_7.json',orient="records")

In [43]:
df.to_csv(r'C:\Users\rona_\Desktop\Aug08_7.csv')
df

,City,Country,Latitude,Longitude,Avg Temperature 2020-08-07,Avg Temperature 2020-08-08,Temp_Difference
0,El Pedregal,Mexico,32.0,-114.0,93,97,4
1,La Joya,Mexico,32.0,-114.3,91,93,2
2,La Capital Del Desierto,Mexico,32.0,-114.6,91,95,4
3,El Doctor,Mexico,32.0,-114.9,91,95,4
4,El Pokar,Mexico,32.0,-115.2,91,95,4
...,...,...,...,...,...,...,...
1253,Happy Camp,United States of America,41.9,-123.6,79,84,5
1254,Patrick Creek,United States of America,41.9,-123.9,68,72,4
1255,Douglas Park,United States of America,41.9,-124.2,64,64,0
1256,Crescent City,United States of America,41.9,-124.5,64,64,0
